# Sentiment Analysis of Twitter posts
*Marcin Zabłocki*



## Abstract
The goal of this project was to predict sentiment for the given Twitter post using Python. Sentiment analysis can predict many different emotions attached to the text, but in this report only 3 major were considered: positive, negative and neutral. The training dataset was small (just over 5900 examples) and the data within it was highly skewed, which greatly impacted on the difficulty of building good classifier. After creating a lot of custom features, utilizing both bag-of-words and word2vec representations and applying the Extreme Gradient Boosting algorithm, the classification accuracy at level of 58% was achieved.


## Used Python Libraries
Data was pre-processed using *pandas*, *gensim* and *numpy* libraries and the learning/validating process was built with *scikit-learn*. Plots were created using *plotly*.

In [1]:
from collections import Counter
import nltk
import pandas as pd
from emoticons import EmoticonDetector
import re as regex
import numpy as np
import plotly
from plotly import graph_objs
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from time import time
import gensim
from os import path

# plotly configuration
plotly.offline.init_notebook_mode(connected=True)

dataset_dir = '~/Documents/aaron_assign'

## Notebook code convention
This report was first prepared as a classical Python project using object oriented programming with maintainability in mind. In order to show this project as a Jupyter Notebook, the classes had to be splitted into multiple code-cells. In order to do so, the classes are suffixed with *_PurposeOfThisSnippet* name and they inherit one from another. The final class will be then run and the results will be shown.

## Data source
The input data consisted two CSV files:
`train.csv` (5971 tweets) and `test.csv` (4000 tweets) - one for training and one for testing.
Format of the data was the following (test data didn't contain Category column):


| Id | Category  | Tweet |
|------|------|------|
|   635930169241374720  | neutral | IOS 9 App Transport Security. Mm need to check if my 3rd party network pod supports it |

All tweets are in english, so it simplifies the processing and analysis.


# Data preprocessing
## Loading the data
        

In [2]:
class TwitterData_Initialize():
    data = []
    processed_data = []
    wordlist = []

    data_model = None
    data_labels = None
    is_testing = False
    
    def initialize(self, csv_file, is_testing_set=False, from_cached=None):
        if from_cached is not None:
            self.data_model = pd.read_csv(from_cached)
            return

        self.is_testing = is_testing_set

        if not is_testing_set:
            self.data = pd.read_csv(csv_file, header=0, names=["id", "emotion", "text"])
            self.data = self.data[self.data["emotion"].isin(["positive", "negative", "neutral"])]
        else:
            self.data = pd.read_csv(csv_file, header=0, names=["id", "text"],dtype={"id":"int64","text":"str"},nrows=4000)
            not_null_text = 1 ^ pd.isnull(self.data["text"])
            not_null_id = 1 ^ pd.isnull(self.data["id"])
            self.data = self.data.loc[not_null_id & not_null_text, :]

        self.processed_data = self.data
        self.wordlist = []
        self.data_model = None
        self.data_labels = None

In [3]:
## Take 1.6 million tweets from Sentiment 140 dataset, filter out and make training dataset

It contains the following 6 fields:

target: the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)

ids: The id of the tweet ( 2087)

date: the date of the tweet (Sat May 16 23:58:44 UTC 2009)

flag: The query (lyx). If there is no query, then this value is NO_QUERY.

user: the user that tweeted (robotickilldozr)

text: the text of the tweet (Lyx is cool)

In [267]:
data = pd.read_csv(path.join(dataset_dir, 'training.1600000.processed.noemoticon.csv'), encoding = "ISO-8859-1")
data.columns = ["target", "ids", "date", "flag", "user", "text"]
data = data[["ids", "target", "text"]]

In [154]:
data

,id,emotion,text
0,1467810672,negative,is upset that he can't update his Facebook by ...
1,1467810917,negative,@Kenichan I dived many times for the ball. Man...
2,1467811184,negative,my whole body feels itchy and like its on fire
3,1467811193,negative,"@nationwideclass no, it's not behaving at all...."
4,1467811372,negative,@Kwesidei not the whole crew
5,1467811592,negative,Need a hug
6,1467811594,negative,@LOLTrish hey long time no see! Yes.. Rains a...
7,1467811795,negative,@Tatiana_K nope they didn't have it
8,1467812025,negative,@twittera que me muera ?
9,1467812416,negative,spring break in plain city... it's snowing


In [153]:
data.columns = ["id", "emotion", "text"]
def int_to_label(x):
    if x == 0:
        return 'negative'
    if x == 2:
        return 'neutral'
    if x == 4:
        return 'positive'
    
data['emotion'] = data[['emotion']].applymap(int_to_label)
data[::16].to_csv('data/train16.csv', sep=',', header=False, index=False)

The code snippet above is prepared, to load the data form the given file for further processing, or just read already preprocessed file from the cache.
There's also a distinction between processing testing and training data. As the ```test.csv``` file was full of empty entries, they were removed.
Additional class properties such as data_model, wordlist etc. will be used further.

In [155]:
data = TwitterData_Initialize()
data.initialize("data/train16.csv")
data.processed_data.head(5)

,id,emotion,text
0,1467813137,negative,about to file taxes
1,1467816149,negative,@julieebaby awe i love you too!!!! 1 am here ...
2,1467820835,negative,@tea oh! i'm so sorry i didn't think about th...
3,1467824199,negative,@Starrbby too bad I won't be around I lost my ...
4,1467834227,negative,"@statravelAU just got ur newsletter, those far..."


## Data distribution
First thing that can be done as soon as the data is loaded is to see the data distribution. The training set had the following distribution:


In [156]:
df = data.processed_data
neg = len(df[df["emotion"] == "negative"])
pos = len(df[df["emotion"] == "positive"])
neu = len(df[df["emotion"] == "neutral"])
dist = [
    graph_objs.Bar(
        x=["negative","neutral","positive"],
        y=[neg, neu, pos],
)]
plotly.offline.iplot({"data":dist, "layout":graph_objs.Layout(title="Sentiment type distribution in training set")})

## Preprocessing steps
The targed of the following preprocessing is to create a **Bag-of-Words** representation of the data. The steps will execute as follows:
1. Cleansing
<ol style="list-style-type:decimal"><li>Remove URLs</li>
<li>Remove usernames (mentions)</li>
<li>Remove tweets with *Not Available* text</li>
<li>Remove special characters</li>
<li>Remove numbers</li></ol>
1. Text processing
<ol style="list-style-type:decimal">
<li>Tokenize</li>
<li>Transform to lowercase</li>
<li>Stem</li></ol>
1. Build word list for Bag-of-Words

### Cleansing
For the purpose of cleansing, the ```TwitterCleanup``` class was created. It consists methods allowing to execute all of the tasks show in the list above. Most of those is done using regular expressions.
The class exposes it's interface through ```iterate()``` method - it yields every cleanup method in proper order.

In [4]:
class TwitterCleanuper:
    def iterate(self):
        for cleanup_method in [self.remove_urls,
                               self.remove_usernames,
                               self.remove_na,
                               self.remove_special_chars,
                               self.remove_numbers]:
            yield cleanup_method

    @staticmethod
    def remove_by_regex(tweets, regexp):
        tweets.loc[:, "text"].replace(regexp, "", inplace=True)
        return tweets

    def remove_urls(self, tweets):
        return TwitterCleanuper.remove_by_regex(tweets, regex.compile(r"http.?://[^\s]+[\s]?"))

    def remove_na(self, tweets):
        return tweets[tweets["text"] != "Not Available"]

    def remove_special_chars(self, tweets):  # it unrolls the hashtags to normal words
        for remove in map(lambda r: regex.compile(regex.escape(r)), [",", ":", "\"", "=", "&", ";", "%", "$",
                                                                     "@", "%", "^", "*", "(", ")", "{", "}",
                                                                     "[", "]", "|", "/", "\\", ">", "<", "-",
                                                                     "!", "?", ".", "'",
                                                                     "--", "---", "#"]):
            tweets.loc[:, "text"].replace(remove, "", inplace=True)
        return tweets

    def remove_usernames(self, tweets):
        return TwitterCleanuper.remove_by_regex(tweets, regex.compile(r"@[^\s]+[\s]?"))

    def remove_numbers(self, tweets):
        return TwitterCleanuper.remove_by_regex(tweets, regex.compile(r"\s?[0-9]+\.?[0-9]*"))

The loaded tweets can be now cleaned. 

In [5]:
class TwitterData_Cleansing(TwitterData_Initialize):
    def __init__(self, previous):
        self.processed_data = previous.processed_data
        
    def cleanup(self, cleanuper):
        t = self.processed_data
        for cleanup_method in cleanuper.iterate():
            if not self.is_testing:
                t = cleanup_method(t)
            else:
                if cleanup_method.__name__ != "remove_na":
                    t = cleanup_method(t)

        self.processed_data = t

In [159]:
data = TwitterData_Cleansing(data)
data.cleanup(TwitterCleanuper())
data.processed_data.head(5)

,id,emotion,text
0,1467813137,negative,about to file taxes
1,1467816149,negative,awe i love you too am here i miss you
2,1467820835,negative,oh im so sorry i didnt think about that befor...
3,1467824199,negative,too bad I wont be around I lost my job and can...
4,1467834227,negative,just got ur newsletter those fares really are ...


### Tokenization & stemming
For the text processing, ```nltk``` library is used. First, the tweets are tokenized using ```nlkt.word_tokenize``` and then, stemming is done using **PorterStemmer** as the tweets are 100% in english.


In [6]:
class TwitterData_TokenStem(TwitterData_Cleansing):
    def __init__(self, previous):
        self.processed_data = previous.processed_data
        
    def stem(self, stemmer=nltk.PorterStemmer()):
        def stem_and_join(row):
            row["text"] = list(map(lambda str: stemmer.stem(str.lower()), row["text"]))
            return row

        self.processed_data = self.processed_data.apply(stem_and_join, axis=1)

    def tokenize(self, tokenizer=nltk.word_tokenize):data = TwitterData_Initialize()
data.initialize("data/train16.csv")
data = TwitterData_Cleansing(data)
data.cleanup(TwitterCleanuper())
data = TwitterData_Wordlist(data)
data.build_wordlist()
        def tokenize_row(row):
            row["text"] = tokenizer(row["text"])
            row["tokenized_text"] = [] + row["text"]
            return row

        self.processed_data = self.processed_data.apply(tokenize_row, axis=1)


In [ ]:
import nltk
nltk.download('punkt')

In [162]:
data = TwitterData_TokenStem(data)
data.tokenize()
data.stem()
data.processed_data.head(5)

,id,emotion,text,tokenized_text
0,1467813137,negative,"[about, to, file, tax]","[about, to, file, taxes]"
1,1467816149,negative,"[awe, i, love, you, too, am, here, i, miss, you]","[awe, i, love, you, too, am, here, i, miss, you]"
2,1467820835,negative,"[oh, im, so, sorri, i, didnt, think, about, th...","[oh, im, so, sorry, i, didnt, think, about, th..."
3,1467824199,negative,"[too, bad, i, wont, be, around, i, lost, my, j...","[too, bad, I, wont, be, around, I, lost, my, j..."
4,1467834227,negative,"[just, got, ur, newslett, those, fare, realli,...","[just, got, ur, newsletter, those, fares, real..."


### Building the wordlist
The wordlist (dictionary) is build by simple count of occurences of every unique word across all of the training dataset.

Before building the final wordlist for the model, let's take a look at the non-filtered version:


In [163]:
words = Counter()
for idx in data.processed_data.index:
    words.update(data.processed_data.loc[idx, "text"])

words.most_common(5)

[('i', 47319), ('to', 35180), ('the', 32192), ('a', 23709), ('it', 20965)]

The most commont words (as expected) are the typical english stopwords. We will filter them out, however, as purpose of this analysis is to determine sentiment, words like "not" and "n't" can influence it greatly. Having this in mind, this word will be whitelisted.


In [ ]:
import nltk
nltk.download('stopwords')

In [165]:
stopwords=nltk.corpus.stopwords.words("english")
whitelist = ["n't", "not"]
for idx, stop_word in enumerate(stopwords):
    if stop_word not in whitelist:
        del words[stop_word]
words.most_common(5)

[('im', 11097), ('go', 8598), ('get', 6750), ('not', 6594), ('wa', 6560)]

Still, there are some words that seem too be occuring to many times, let's filter them. After some analysis, the lower bound was set to 3.

The wordlist is also saved to the csv file, so the same words can be used for the testing set.


In [171]:
words["thank"]

3579

In [166]:
data_before_word_gen = data

In [167]:
%store data_before_word_gen

Proper storage of interactively declared classes (or instances
of those classes) is not possible! Only instances
of classes in real modules on file system can be %store'd.



In [168]:
%store -r data_before_word_gen

no stored variable data_before_word_gen


In [20]:
data_before_word_gen.data

[]

In [7]:
class TwitterData_Wordlist(TwitterData_TokenStem):
    def __init__(self, previous):
        self.processed_data = previous.processed_data
        
    whitelist = ["n't","not"]
    wordlist = []
        
    def build_wordlist(self, min_occurrences=100, max_occurences=np.inf, stopwords=nltk.corpus.stopwords.words("english"),
                       whitelist=None):
        self.wordlist = []
        whitelist = self.whitelist if whitelist is None else whitelist
        import os
        if os.path.isfile("data/wordlist.csv"):
           word_df = pd.read_csv("data/wordlist.csv")
           word_df = word_df[word_df["occurrences"] > min_occurrences]
           self.wordlist = list(word_df.loc[:, "word"])
           return

        words = Counter()
        for idx in self.processed_data.index:
            words.update(self.processed_data.loc[idx, "text"])

        for idx, stop_word in enumerate(stopwords):
            if stop_word not in whitelist:
                del words[stop_word]

        word_df = pd.DataFrame(data={"word": [k for k, v in words.most_common() if min_occurrences < v < max_occurences],
                                     "occurrences": [v for k, v in words.most_common() if min_occurrences < v < max_occurences]},
                               columns=["word", "occurrences"])

        word_df.to_csv("data/wordlist.csv", index_label="idx")
        self.wordlist = [k for k, v in words.most_common() if min_occurrences < v < max_occurences]

In [204]:
data = TwitterData_Wordlist(data)
data.build_wordlist()

In [205]:
'thank' in data.wordlist

True

In [206]:
len(data.wordlist)

988

In [207]:
words = pd.read_csv("data/wordlist.csv")
x_words = list(words.loc[0:10,"word"])
x_words.reverse()
y_occ = list(words.loc[0:10,"occurrences"])
y_occ.reverse()

dist = [
    graph_objs.Bar(
        x=y_occ,
        y=x_words,
        orientation="h"
)]
plotly.offline.iplot({"data":dist, "layout":graph_objs.Layout(title="Top words in built wordlist")})

### Bag-of-words
The data is ready to transform it to bag-of-words representation.


In [120]:
class TwitterData_BagOfWords(TwitterData_Wordlist):
    def __init__(self, previous):
        self.processed_data = previous.processed_data
        self.wordlist = previous.wordlist
    
    def build_data_model(self):
        label_column = []
        if not self.is_testing:
            label_column = ["label"]

        columns = label_column + list(
            map(lambda w: w + "_bow",self.wordlist))
        labels = []
        rows = []
        for idx in self.processed_data.index:
            current_row = []

            if not self.is_testing:
                # add label
                current_label = self.processed_data.loc[idx, "emotion"]
                labels.append(current_label)
                current_row.append(current_label)

            # add bag-of-words
            tokens = set(self.processed_data.loc[idx, "text"])
            for _, word in enumerate(self.wordlist):
                current_row.append(1 if word in tokens else 0)

            rows.append(current_row)

        self.data_model = pd.DataFrame(rows, columns=columns)
        self.data_labels = pd.Series(labels)
        return self.data_model, self.data_labels

Let's take a look at the data and see, which words are the most common for particular sentiments.

In [89]:
data = TwitterData_BagOfWords(data)

bow, labels = data.build_data_model()
bow.head(5)

In [91]:
grouped = bow.groupby(["label"]).sum()
grouped.index

Index(['negative', 'positive'], dtype='object', name='label')

In [92]:
grouped = bow.groupby(["label"]).sum()
words_to_visualize = []
sentiments = ["positive","negative","neutral"]
#get the most 7 common words for every sentiment
for sentiment in sentiments:
    if sentiment not in grouped.index:
        continue
    words = grouped.loc[sentiment,:]
    words.sort_values(inplace=True,ascending=False)
    for w in words.index[:7]:
        if w not in words_to_visualize:
            words_to_visualize.append(w)
            
            
#visualize it
plot_data = []
for sentiment in sentiments:
    if sentiment not in grouped.index:
        continue
    plot_data.append(graph_objs.Bar(
            x = [w.split("_")[0] for w in words_to_visualize],
            y = [grouped.loc[sentiment,w] for w in words_to_visualize],
            name = sentiment
    ))

plotly.offline.iplot({
        "data":plot_data,
        "layout":graph_objs.Layout(title="Most common words across sentiments")
    })
    


Some of the most common words show high distinction between classes like *go* and *see* and other are occuring in similiar amount for every class (*plan*, *obama*).

None of the most common words is unique to the negative class. At this point, it's clear that skewed data distribution will be a problem in distinguishing negative tweets from the others. 

# Classification
First of all, lets establish seed for random numbers generators.

In [94]:
import random
seed = 666
random.seed(seed)

The following utility function will train the classifier and show the F1, precision, recall and accuracy scores.

In [95]:
def test_classifier(X_train, y_train, X_test, y_test, classifier):
    log("")
    log("===============================================")
    classifier_name = str(type(classifier).__name__)
    log("Testing " + classifier_name)
    now = time()
    list_of_labels = sorted(list(set(y_train)))
    model = classifier.fit(X_train, y_train)
    log("Learing time {0}s".format(time() - now))
    now = time()
    predictions = model.predict(X_test)
    log("Predicting time {0}s".format(time() - now))

    precision = precision_score(y_test, predictions, average=None, pos_label=None, labels=list_of_labels)
    recall = recall_score(y_test, predictions, average=None, pos_label=None, labels=list_of_labels)
    accuracy = accuracy_score(y_test, predictions)
    f1 = f1_score(y_test, predictions, average=None, pos_label=None, labels=list_of_labels)
    log("=================== Results ===================")
    log("            Negative     Neutral     Positive")
    log("F1       " + str(f1))
    log("Precision" + str(precision))
    log("Recall   " + str(recall))
    log("Accuracy " + str(accuracy))
    log("===============================================")

    return precision, recall, accuracy, f1

def log(x):
    #can be used to write to log file
    print(x)

## Experiment 1: BOW + Naive Bayes
It is nice to see what kind of results we might get from such simple model. The bag-of-words representation is binary, so Naive Bayes Classifier seems like a nice algorithm to start the experiments.

The experiment will be based on ```7:3``` train:test stratified split.



In [96]:
from sklearn.naive_bayes import BernoulliNB
X_train, X_test, y_train, y_test = train_test_split(bow.iloc[:, 1:], bow.iloc[:, 0],
                                                    train_size=0.7, stratify=bow.iloc[:, 0],
                                                    random_state=seed)
precision, recall, accuracy, f1 = test_classifier(X_train, y_train, X_test, y_test, BernoulliNB())

/home/akvasov/anaconda3/envs/sen_anal/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning:

From version 0.21, test_size will always complement train_size unless both are specified.




Testing BernoulliNB
Learing time 2.4626076221466064s
Predicting time 0.9015510082244873s
=================== Results ===================
            Negative     Neutral     Positive
F1       [0.73891254 0.75359596]
Precision[0.76160487 0.73298462]
Recall   [0.71753333 0.7754    ]
Accuracy 0.7464666666666666


Result with accuracy at level of 58% seems to be quite nice result for such basic algorithm like Naive Bayes (having in mind that random classifier would yield result of around 33% accuracy). This performance may not hold for the final testing set. In order to see how the NaiveBayes performs in more general cases, 8-fold crossvalidation is used. The 8 fold is used, to optimize speed of testing on my 8-core machine.


In [11]:
def cv(classifier, X_train, y_train):
    log("===============================================")
    classifier_name = str(type(classifier).__name__)
    now = time()
    log("Crossvalidating " + classifier_name + "...")
    accuracy = [cross_val_score(classifier, X_train, y_train, cv=8, n_jobs=-1)]
    log("Crosvalidation completed in {0}s".format(time() - now))
    log("Accuracy: " + str(accuracy[0]))
    log("Average accuracy: " + str(np.array(accuracy[0]).mean()))
    log("===============================================")
    return accuracy

In [218]:
nb_acc = cv(BernoulliNB(), bow.iloc[:,1:], bow.iloc[:,0])

Crossvalidating BernoulliNB...
Crosvalidation completed in 14.390513896942139s
Accuracy: [0.74024    0.7324     0.74488    0.75272    0.74304    0.74392
 0.74432    0.74461957]
Average accuracy: 0.7432674461956956


This result no longer looks optimistic. For some of the splits, Naive Bayes classifier showed performance below the performance of random classifier.

### Testing on our 100 messages dataset for "gene editing topic" reference

In [113]:
X_train, X_test, y_train, y_test = train_test_split(bow.iloc[:, 1:], bow.iloc[:, 0],
                                                    train_size=0.7, stratify=bow.iloc[:, 0],
                                                    random_state=seed)
classifier = BernoulliNB()

model = classifier.fit(X_train, y_train)
predictions = model.predict(X_test)

/home/akvasov/anaconda3/envs/sen_anal/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning:

From version 0.21, test_size will always complement train_size unless both are specified.



In [ ]:
data = TwitterData_BagOfWords(data)

bow, labels = data.build_data_model()

In [141]:
data = TwitterData()
data.initialize(path.join(dataset_dir, "filtered_gene_testdataset.csv"))
#data.build_features()
data.cleanup(TwitterCleanuper())
data.tokenize()
data.stem()
data.build_wordlist()
data.build_data_model()
test_data = data

In [142]:
list_of_labels = sorted(list(set(y_train)))

X_test = test_data.data_model.iloc[:,1:]
y_test = test_data.data_model.iloc[:,0]
predictions = model.predict(X_test)

precision = precision_score(y_test, predictions, average=None, pos_label=None, labels=list_of_labels)
recall = recall_score(y_test, predictions, average=None, pos_label=None, labels=list_of_labels)
accuracy = accuracy_score(y_test, predictions)
f1 = f1_score(y_test, predictions, average=None, pos_label=None, labels=list_of_labels)
log("=================== Results ===================")
log("            Negative     Neutral     Positive")
log("F1       " + str(f1))
log("Precision" + str(precision))
log("Recall   " + str(recall))
log("Accuracy " + str(accuracy))
log("===============================================")

=================== Results ===================
            Negative     Neutral     Positive
F1       [0.66666667 0.83333333]
Precision[0.75862069 0.78571429]
Recall   [0.59459459 0.88709677]
Accuracy 0.7777777777777778


## Additional features
In order to **not** push any other aglorithm to the limit on the current data model, let's try to add some features that might help to classify tweets.

A common sense suggest that special characters like exclamation marks and the casing might be important in the task of determining the sentiment.
The following features will be added to the data model:

| Feature name | Explanation |
|------|------|
|Number of uppercase | people tend to express with either positive or negative emotions by using A LOT OF UPPERCASE WORDS |
| Number of ! | exclamation marks are likely to increase the strength of opinion |
| Number of ? | might distinguish neutral tweets - seeking for information |
| Number of positive emoticons | positive emoji will most likely not occur in the negative tweets|
| Number of negative emoticons | inverse to the one above |
| Number of ... | commonly used in commenting something |
| Number of quotations | same as above | 
| Number of mentions | sometimes people put a lot of mentions on positive tweets, to share something good |
| Number of hashtags | just for the experiment |
| Number of urls | similiar to the number of mentions |

Extraction of those features must be done before any preprocessing happens.

For the purpose of emoticons, the ```EmoticonDetector``` class is created. The file ```emoticons.txt``` contains list of positive and negative emoticons, which are used.

In [23]:
class EmoticonDetector:
    emoticons = {}

    def __init__(self, emoticon_file="data/emoticons.txt"):
        from pathlib import Path
        content = Path(emoticon_file).read_text()
        positive = True
        for line in content.split("\n"):
            if "positive" in line.lower():
                positive = True
                continue
            elif "negative" in line.lower():
                positive = False
                continue

            self.emoticons[line] = positive

    def is_positive(self, emoticon):
        if emoticon in self.emoticons:
            return self.emoticons[emoticon]
        return False

    def is_emoticon(self, to_check):
        return to_check in self.emoticons

In [24]:
class TwitterData_ExtraFeatures(TwitterData_Wordlist):
    def __init__(self):
        pass
    
    def build_data_model(self):
        extra_columns = [col for col in self.processed_data.columns if col.startswith("number_of")]
        label_column = []
        if not self.is_testing:
            label_column = ["label"]

        columns = label_column + extra_columns + list(
            map(lambda w: w + "_bow",self.wordlist))
        
        labels = []
        rows = []
        for idx in self.processed_data.index:
            current_row = []

            if not self.is_testing:
                # add label
                current_label = self.processed_data.loc[idx, "emotion"]
                labels.append(current_label)
                current_row.append(current_label)

            for _, col in enumerate(extra_columns):
                current_row.append(self.processed_data.loc[idx, col])

            # add bag-of-words
            tokens = set(self.processed_data.loc[idx, "text"])
            for _, word in enumerate(self.wordlist):
                current_row.append(1 if word in tokens else 0)

            rows.append(current_row)

        self.data_model = pd.DataFrame(rows, columns=columns)
        self.data_labels = pd.Series(labels)
        return self.data_model, self.data_labels
    
    def build_features(self):
        def count_by_lambda(expression, word_array):
            return len(list(filter(expression, word_array)))

        def count_occurences(character, word_array):
            counter = 0
            for j, word in enumerate(word_array):
                for char in word:
                    if char == character:
                        counter += 1

            return counter

        def count_by_regex(regex, plain_text):
            return len(regex.findall(plain_text))

        self.add_column("splitted_text", map(lambda txt: txt.split(" "), self.processed_data["text"]))

        # number of uppercase words
        uppercase = list(map(lambda txt: count_by_lambda(lambda word: word == word.upper(), txt),
                             self.processed_data["splitted_text"]))
        self.add_column("number_of_uppercase", uppercase)

        # number of !
        exclamations = list(map(lambda txt: count_occurences("!", txt),
                                self.processed_data["splitted_text"]))

        self.add_column("number_of_exclamation", exclamations)

        # number of ?
        questions = list(map(lambda txt: count_occurences("?", txt),
                             self.processed_data["splitted_text"]))

        self.add_column("number_of_question", questions)

        # number of ...
        ellipsis = list(map(lambda txt: count_by_regex(regex.compile(r"\.\s?\.\s?\."), txt),
                            self.processed_data["text"]))

        self.add_column("number_of_ellipsis", ellipsis)

        # number of hashtags
        hashtags = list(map(lambda txt: count_occurences("#", txt),
                            self.processed_data["splitted_text"]))

        self.add_column("number_of_hashtags", hashtags)

        # number of mentions
        mentions = list(map(lambda txt: count_occurences("@", txt),
                            self.processed_data["splitted_text"]))

        self.add_column("number_of_mentions", mentions)

        # number of quotes
        quotes = list(map(lambda plain_text: int(count_occurences("'", [plain_text.strip("'").strip('"')]) / 2 +
                                                 count_occurences('"', [plain_text.strip("'").strip('"')]) / 2),
                          self.processed_data["text"]))

        self.add_column("number_of_quotes", quotes)

        # number of urls
        urls = list(map(lambda txt: count_by_regex(regex.compile(r"http.?://[^\s]+[\s]?"), txt),
                        self.processed_data["text"]))

        self.add_column("number_of_urls", urls)

        # number of positive emoticons
        ed = EmoticonDetector()
        positive_emo = list(
            map(lambda txt: count_by_lambda(lambda word: ed.is_emoticon(word) and ed.is_positive(word), txt),
                self.processed_data["splitted_text"]))

        self.add_column("number_of_positive_emo", positive_emo)

        # number of negative emoticons
        negative_emo = list(map(
            lambda txt: count_by_lambda(lambda word: ed.is_emoticon(word) and not ed.is_positive(word), txt),
            self.processed_data["splitted_text"]))

        self.add_column("number_of_negative_emo", negative_emo)
        
    def add_column(self, column_name, column_content):
        self.processed_data.loc[:, column_name] = pd.Series(column_content, index=self.processed_data.index)


In [25]:
data = TwitterData_ExtraFeatures()
data.initialize("data/train16.csv")
data.build_features()
data.cleanup(TwitterCleanuper())
data.tokenize()
data.stem()
data.build_wordlist()
data_model, labels = data.build_data_model()
data_model.head(5)

,label,number_of_uppercase,number_of_exclamation,number_of_question,number_of_ellipsis,number_of_hashtags,number_of_mentions,number_of_quotes,number_of_urls,number_of_positive_emo,...,bill_bow,expens_bow,bank_bow,prob_bow,quotth_bow,honey_bow,choic_bow,group_bow,trek_bow,chri_bow
0,negative,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,negative,2,4,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,negative,1,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,negative,3,0,0,0,0,1,1,0,0,...,1,0,0,0,0,0,0,0,0,0
4,negative,2,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Logic behind extra features
Let's see how (some) of the extra features separate the data set. Some of them, i.e number exclamation marks, number of pos/neg emoticons do this really well. Despite of the good separation, those features sometimes occur only on small subset of the training dataset.

In [26]:
sentiments = ["positive","negative","neutral"]
plots_data_ef = []
for what in map(lambda o: "number_of_"+o,["positive_emo","negative_emo","exclamation","hashtags","question"]):
    ef_grouped = data_model[data_model[what]>=1].groupby(["label"]).count()
    plots_data_ef.append({"data":[graph_objs.Bar(
            x = sentiments,
            y = [ef_grouped.loc[s,:][0] for s in sentiments if s in ef_grouped.index],
    )], "title":"How feature \""+what+"\" separates the tweets"})
    

for plot_data_ef in plots_data_ef:
    plotly.offline.iplot({
            "data":plot_data_ef["data"],
            "layout":graph_objs.Layout(title=plot_data_ef["title"])
    })


## Experiment 2: extended features + Random Forest
As a second attempt on the classification the **Random Forest** will be used.

In [27]:
from sklearn.ensemble import RandomForestClassifier
X_train, X_test, y_train, y_test = train_test_split(data_model.iloc[:, 1:], data_model.iloc[:, 0],
                                                    train_size=0.7, stratify=data_model.iloc[:, 0],
                                                    random_state=seed)
precision, recall, accuracy, f1 = test_classifier(X_train, y_train, X_test, y_test, RandomForestClassifier(random_state=seed,n_estimators=403,n_jobs=-1))

/home/akvasov/anaconda3/envs/sen_anal/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning:

From version 0.21, test_size will always complement train_size unless both are specified.




Testing RandomForestClassifier
Learing time 161.3562970161438s
Predicting time 3.748042106628418s
=================== Results ===================
            Negative     Neutral     Positive
F1       [0.74481796 0.74304255]
Precision[0.74225371 0.74563641]
Recall   [0.7474     0.74046667]
Accuracy 0.7439333333333333


The accuracy for the initial split was lower than the one for the Naive Bayes, but let's see what happens during crossvalidation:


In [29]:
rf_acc = cv(RandomForestClassifier(n_estimators=403,n_jobs=-1, random_state=seed),data_model.iloc[:, 1:], data_model.iloc[:, 0])

Crossvalidating RandomForestClassifier...
Crosvalidation completed in 70.09595036506653s
Accuracy: [ 0.54344624  0.50442478  0.37905605  0.27876106  0.37905605  0.52141802
  0.5155096   0.55621302]
Average accuracy: 0.459735602399


It looks better, however it's still not much above accuracy of the random classifier and barely better than Naive Bayes classifier.

We can observe a low recall level of the RandomForest classifier for the negative class, which may be caused by the data skewness.

## More features - word2vec
The overall performance of the previous classifiers could be enhanced by performing time-consuming parameters adjustments, however there's not guarantee on how big the gain will be.

If the out-of-the-shelf methods did not performed well, it seems that there's not much in the data itself. The next idea to add more into data model is to use word2vec representation of a tweet to perform classification. 

The word2vec allows to transform words into vectors of numbers. Those vectors represent abstract features, that describe the word similarities and relationships (i.e co-occurence).

What is the best in the word2vec is that operations on the vectors approximately keep the characteristics of the words, so that joining (averaging) vectors from the words from sentence procude vector that is likely to represent the general topic of the sentence.

A lot of pre-trained word2vec models exists, and some of them were trained on huge volumes of data. For the purpose of this analysis, the one trained on over **2 billion of tweets** with 200 dimensions (one vector consists of 200 numbers) is used.
The pre-trained model can be downloaded here: https://github.com/3Top/word2vec-api


### From GloVe to word2vec
In order to use GloVe-trained model in ```gensim``` library, it needs to be converted to word2vec format. The only difference between those formats is that word2vec text files starts with two numbers: *number of lines in file* and *number of dimensions*. The file ```glove.twitter.27B.200d.txt``` does not contain those lines.

Unfortunaltely, this text file size is over 1.9GB and text editors cannot be used to open and modify it in reasonable amount of time, this **C#** snippet adds this required line (sorry that it's not Python, but I was having memory problems with encoding of the file in Python. It's required to use x64 target):
```{Csharp}
using (var fileStream = new FileStream("glove.twitter.27B.200d.txt", FileMode.Open,FileAccess.ReadWrite))
{
    var lines = new LinkedList<string>();
    using (var streamReader = new StreamReader(fileStream))
    {
        while (!streamReader.EndOfStream)
        {
            lines.AddLast(streamReader.ReadLine());
        }
    }
    lines.AddFirst("1193514 200");
    File.WriteAllLines("word2vec.twitter.27B.200d.txt.txt", lines);
}
```

#### Already modified GloVe for file
The file that has the first line appended can be downloaded from here (622MB 7-zip file, ultra compression): https://marcin.egnyte.com/dl/gk9nRsVMMY

### Using Word2Vec
The following class exposes a easy to use interface over the word2vec API from ```gensim``` library:



In [36]:
import gluonnlp as nlp

In [37]:

glove[["hello", "world"]]


[[-0.38497   0.80092   0.064106 -0.28355  -0.026759 -0.34532  -0.64253
  -0.11729  -0.33257   0.55243  -0.087813  0.9035    0.47102   0.56657
   0.6985   -0.35229  -0.86542   0.90573   0.03576  -0.071705 -0.12327
   0.54923   0.47005   0.35572   1.2611   -0.67581  -0.94983   0.68666
   0.3871   -1.3492    0.63512   0.46416  -0.48814   0.83827  -0.9246
  -0.33722   0.53741  -1.0616   -0.081403 -0.67111   0.30923  -0.3923
  -0.55002  -0.68827   0.58049  -0.11626   0.013139 -0.57654   0.048833
   0.67204 ]
 [-0.41486   0.71848  -0.3045    0.87445   0.22441  -0.56488  -0.37566
  -0.44801   0.61347  -0.11359   0.74556  -0.10598  -1.1882    0.50974
   1.3511    0.069851  0.73314   0.26773  -1.1787   -0.148     0.039853
   0.033107 -0.27406   0.25125   0.41507  -1.6188   -0.81778  -0.73892
  -0.28997   0.57277   3.4719    0.73817  -0.044495 -0.15119  -0.93503
  -0.13152  -0.28562   0.76327  -0.83332  -0.6793   -0.39099  -0.64466
   1.0044   -0.2051    0.46799   0.99314  -0.16221  -0.46022  -

In [17]:
class Word2VecProvider(object):
    word2vec = None
    dimensions = 0

    def load(self, path_to_word2vec):
        self.word2vec = gensim.models.KeyedVectors.load_word2vec_format(path_to_word2vec, binary=False)
        self.word2vec.init_sims(replace=True)
        self.dimensions = self.word2vec.vector_size

    def get_vector(self, word):
        if word not in self.word2vec.vocab:
            return None

        return self.word2vec.syn0norm[self.word2vec.vocab[word].index]

    def get_similarity(self, word1, word2):
        if word1 not in self.word2vec.vocab or word2 not in self.word2vec.vocab:
            return None

        return self.word2vec.similarity(word1, word2)

In [18]:
word2vec = Word2VecProvider()
model_dir = path.join(dataset_dir, "glove.twitter.27B/glove.twitter.27B/glove.twitter.27B.25d.txt")
# REPLACE PATH TO THE FILE
word2vec.load(model_dir)

### Extra features from word2vec
Besides the 200 additional features from the word2vec representation, I had an idea of 3 more features. If word2vec allows to find similarity between words, that means it can find similarity to the specific emotion-representing words. The first idea was to compute similarity of the whole tweet with words from labels: *positive, negative, neutral*. Since the purpose was to find the sentiment, I thought that it will be better to find similarity with more expressive words such as: **good** and **bad**. For the neutral sentiment, I've used word **information**, since most of the tweets with neutral sentiment were giving the information.

The features were builded by computing **mean similarity of the whole tweet to the given word**. Then, those mean values were normalized to [0;1] in order to deal with different word count across tweets.

## Final data model
The final data model will contain:
* extra text features (number of: !, ?, :-) etc)
* word2vec similarity to "good", "bad" and "information" words
* word2vec 200 dimension averaged representation of a tweet
* bag-of-word representation of a tweet


In [15]:
class TwitterData(TwitterData_ExtraFeatures):
    
    def build_final_model(self, word2vec_provider, stopwords=nltk.corpus.stopwords.words("english")):
        whitelist = self.whitelist
        stopwords = list(filter(lambda sw: sw not in whitelist, stopwords))
        extra_columns = [col for col in self.processed_data.columns if col.startswith("number_of")]
        similarity_columns = ["bad_similarity", "good_similarity", "information_similarity"]
        label_column = []
        if not self.is_testing:
            label_column = ["label"]

        columns = label_column + ["original_id"] + extra_columns + similarity_columns + list(
            map(lambda i: "word2vec_{0}".format(i), range(0, word2vec_provider.dimensions))) + list(
            map(lambda w: w + "_bow",self.wordlist))
        labels = []
        rows = []
        for idx in self.processed_data.index:
            current_row = []

            if not self.is_testing:
                # add label
                current_label = self.processed_data.loc[idx, "emotion"]
                labels.append(current_label)
                current_row.append(current_label)

            current_row.append(self.processed_data.loc[idx, "id"])

            for _, col in enumerate(extra_columns):
                current_row.append(self.processed_data.loc[idx, col])

            # average similarities with words
            tokens = self.processed_data.loc[idx, "tokenized_text"]
            for main_word in map(lambda w: w.split("_")[0], similarity_columns):
                current_similarities = [abs(sim) for sim in
                                        map(lambda word: word2vec_provider.get_similarity(main_word, word.lower()), tokens) if
                                        sim is not None]
                if len(current_similarities) <= 1:
                    current_row.append(0 if len(current_similarities) == 0 else current_similarities[0])
                    continue
                max_sim = max(current_similarities)
                min_sim = min(current_similarities)
                if max_sim == min_sim:
                    current_row.append(max_sim)
                    continue
                current_similarities = [((sim - min_sim) / (max_sim - min_sim)) for sim in
                                        current_similarities]  # normalize to <0;1>
                current_row.append(np.array(current_similarities).mean())

            # add word2vec vector
            tokens = self.processed_data.loc[idx, "tokenized_text"]
            current_word2vec = []
            for _, word in enumerate(tokens):
                vec = word2vec_provider.get_vector(word.lower())
                if vec is not None:
                    current_word2vec.append(vec)

            averaged_word2vec = np.array(current_word2vec).mean(axis=0).tolist()
            if np.any(np.isnan(averaged_word2vec)):
                averaged_word2vec = np.zeros((word2vec_provider.dimensions,)).tolist()
                
            current_row += averaged_word2vec

            # add bag-of-words
            tokens = set(self.processed_data.loc[idx, "text"])
            for _, word in enumerate(self.wordlist):
                current_row.append(1 if word in tokens else 0)

            rows.append(current_row)

        self.data_model = pd.DataFrame(rows, columns=columns)
        self.data_labels = pd.Series(labels)
        return self.data_model, self.data_labels

In [36]:
td = TwitterData()
td.initialize("data/train16.csv")
td.build_features()
td.cleanup(TwitterCleanuper())
td.tokenize()
td.stem()
td.build_wordlist()

In [ ]:
td.build_final_model(word2vec)

td.data_model.head(5)

In [108]:
data_model = td.data_model
data_model.drop("original_id",axis=1,inplace=True)


In [39]:
data_before_word_gen = data_model
% store data_before_after_word2vec

UsageError: Line magic function `%` not found.


### Logic behind word2vec extra features
In order to show, why the 3 extra features built using word2vec might help in sentiment analysis. The chart below shows how many tweets from given category were dominating (had highest value) on similarity to those words.
Although the words doesn't seem to separate the sentiments themselves, the differences between them in addition to other parameters, may help the classification process - i.e when tweet has highest value on *good_similarity* it's more likely for it to be classified to have positive sentiment.


In [259]:
columns_to_plot = ["bad_similarity", "good_similarity", "information_similarity"]
bad, good, info = columns_to_plot
sentiments = ["positive","negative","neutral"]
only_positive = data_model[data_model[good]>=data_model[bad]]
only_positive = only_positive[only_positive[good]>=only_positive[info]].groupby(["label"]).count()

only_negative = data_model[data_model[bad] >= data_model[good]]
only_negative = only_negative[only_negative[bad] >= only_negative[info]].groupby(["label"]).count()

only_info = data_model[data_model[info]>=data_model[good]]
only_info = only_info[only_info[info]>=only_info[bad]].groupby(["label"]).count()

plot_data_w2v = []
for sentiment in sentiments:
    if sentiment not in grouped.index:
        continue
    plot_data_w2v.append(graph_objs.Bar(
            x = ["good","bad", "information"],
            y = [only_positive.loc[sentiment,:][0], only_negative.loc[sentiment,:][0], only_info.loc[sentiment,:][0]],
            name = "Number of dominating " + sentiment
    ))
    
plotly.offline.iplot({
        "data":plot_data_w2v,
        "layout":graph_objs.Layout(title="Number of tweets dominating on similarity to: good, bad, information")
    })

## Experiment 3: full model + Random Forest
The model is now complete. With a lot of new features, the learning algorithms should perform totally differently on the new data set.


In [40]:
X_train, X_test, y_train, y_test = train_test_split(data_model.iloc[:, 1:], data_model.iloc[:, 0],
                                                    train_size=0.7, stratify=data_model.iloc[:, 0],
                                                    random_state=seed)
precision, recall, accuracy, f1 = test_classifier(X_train, y_train, X_test, y_test, RandomForestClassifier(n_estimators=403,n_jobs=-1, random_state=seed))

/home/akvasov/anaconda3/envs/sen_anal/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning:

From version 0.21, test_size will always complement train_size unless both are specified.




Testing RandomForestClassifier
Learing time 66.11012649536133s
Predicting time 1.482553482055664s
=================== Results ===================
            Negative     Neutral     Positive
F1       [0.75139878 0.74797785]
Precision[0.74633344 0.75315985]
Recall   [0.75653333 0.74286667]
Accuracy 0.7497


In [110]:
X_train, X_test, y_train, y_test = train_test_split(data_model.iloc[:, 1:], data_model.iloc[:, 0],
                                                    train_size=0.7, stratify=data_model.iloc[:, 0],
                                                    random_state=seed)
classifier = RandomForestClassifier(n_estimators=403,n_jobs=-1, random_state=seed)

model = classifier.fit(X_train, y_train)
predictions = model.predict(X_test)

/home/akvasov/anaconda3/envs/sen_anal/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning:

From version 0.21, test_size will always complement train_size unless both are specified.



### Feature importance for Random Forest model (like xgboost)

In [45]:
features = {}
for idx, fi in enumerate(classifier.feature_importances_):
    features[data_model.columns[1+idx]] = fi

important = []
for f in sorted(features,key=features.get,reverse=True):
    important.append((f,features[f]))
    print(f + " " + str(features[f]))
    

word2vec_22 0.044264489130073637
word2vec_19 0.03516557381575901
word2vec_9 0.032555193630619386
word2vec_21 0.027603438947028155
word2vec_17 0.026433948439632773
word2vec_15 0.024704563142900673
word2vec_20 0.024522982662411614
word2vec_7 0.023312459786694245
word2vec_0 0.02295057240039318
word2vec_14 0.022262212809801164
word2vec_18 0.02219127241878715
word2vec_2 0.020867482191840776
word2vec_5 0.020805903011582673
word2vec_6 0.020658312254964617
word2vec_3 0.0202814454020409
word2vec_10 0.020214788342514518
word2vec_24 0.02011606374765884
word2vec_8 0.019804521821893836
word2vec_23 0.019731531130739662
word2vec_1 0.0196932258434081
word2vec_13 0.019498597812885408
word2vec_12 0.01933118592441679
word2vec_11 0.019238055086251822
word2vec_4 0.019176764597854692
word2vec_16 0.019059604228721508
bad_similarity 0.0184155080425661
information_similarity 0.018396770810620622
good_similarity 0.018277185962704456
number_of_mentions 0.01737036448624118
number_of_exclamation 0.0099881036724651

In [261]:
rf_acc = cv(RandomForestClassifier(n_estimators=403,n_jobs=-1,random_state=seed),data_model.iloc[:, 1:], data_model.iloc[:, 0])

/home/akvasov/anaconda3/envs/sen_anal/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning:

From version 0.21, test_size will always complement train_size unless both are specified.




Testing RandomForestClassifier
Learing time 61.461753606796265s
Predicting time 1.4276564121246338s
=================== Results ===================
            Negative     Neutral     Positive
F1       [0.75139878 0.74797785]
Precision[0.74633344 0.75315985]
Recall   [0.75653333 0.74286667]
Accuracy 0.7497
Crossvalidating RandomForestClassifier...
Crosvalidation completed in 771.9572877883911s
Accuracy: [0.74504    0.744      0.7532     0.7532     0.75384    0.75064
 0.748      0.74653972]
Average accuracy: 0.7493074653972318


As we can see, the average accuracy from crossvalidation is almost 58%, and the results from the crossvalidation runs are more stable - they never drop below 51%. It might seem that the algorithm will perform pretty well on the testing set. There's a one problem with that - recall for the 7:3 split shows, that only about 3% of the negative tweets from the whole set were recognized properly - it seems like the algorithm is good with recognition of postive vs. neutral cases, but it does really poor job when it comes to recognize negative sentiment. Let's try something else.

### Processing test dataset with "gene editing" related Tweets

In [104]:
test_dataset = pd.read_csv(path.join(dataset_dir, "gene_testdataset.csv"), names=["id", "emotion", "text"])
test_dataset = test_dataset[test_dataset != "neutral"]
test_dataset.groupby("emotion").count()
test_dataset.to_csv(path.join(dataset_dir, "filtered_gene_testdataset.csv"), header=False, index=False)

In [105]:
test_data = TwitterData()
test_data.initialize(path.join(dataset_dir, "filtered_gene_testdataset.csv"))
test_data.build_features()
test_data.cleanup(TwitterCleanuper())
test_data.tokenize()
test_data.stem()
test_data.build_wordlist()
test_data.build_final_model(word2vec)
test_data.data_model.drop("original_id",axis=1,inplace=True)

test_data.data_model.head(5)

/home/akvasov/anaconda3/envs/sen_anal/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning:

Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.wv.vectors_norm instead).



,label,number_of_uppercase,number_of_exclamation,number_of_question,number_of_ellipsis,number_of_hashtags,number_of_mentions,number_of_quotes,number_of_urls,number_of_positive_emo,...,bill_bow,expens_bow,bank_bow,prob_bow,quotth_bow,honey_bow,choic_bow,group_bow,trek_bow,chri_bow
0,positive,14,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,positive,13,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,positive,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,negative,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,negative,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [112]:
list_of_labels = sorted(list(set(y_train)))

X_test = test_data.data_model.iloc[:,1:]
y_test = test_data.data_model.iloc[:,0]
predictions = model.predict(X_test)

precision = precision_score(y_test, predictions, average=None, pos_label=None, labels=list_of_labels)
recall = recall_score(y_test, predictions, average=None, pos_label=None, labels=list_of_labels)
accuracy = accuracy_score(y_test, predictions)
f1 = f1_score(y_test, predictions, average=None, pos_label=None, labels=list_of_labels)
log("=================== Results ===================")
log("            Negative     Neutral     Positive")
log("F1       " + str(f1))
log("Precision" + str(precision))
log("Recall   " + str(recall))
log("Accuracy " + str(accuracy))
log("===============================================")

=================== Results ===================
            Negative     Neutral     Positive
F1       [0.61538462 0.75      ]
Precision[0.58536585 0.77586207]
Recall   [0.64864865 0.72580645]
Accuracy 0.696969696969697


# Feature importance
Let's take a look at the final model feature importance.

*Output list is really long, so the print line is commented in the code block*

In [46]:
features = {}
for idx, fi in enumerate(xgboost.feature_importances_):
    features[test_model.columns[1+idx]] = fi

important = []
for f in sorted(features,key=features.get,reverse=True):
    important.append((f,features[f]))
    # print(f + " " + str(features[f]))
    

It's not surprise that the word2vec and related good/bad/information similarity features were the most important, because the classification performance was greatly improved after switching to this representation.

What is interesting is that a lot of custom-crafted features (number\_of\_\*) were also highly important, beating a lot of features which came from bag-of-words representation.
Here's the chart for the easiness of reading:


In [47]:
to_show = list(filter(lambda f: not f[0].startswith("word2vec") and not f[0].endswith("_similarity"),important))[:20]
to_show.reverse()
features_importance = [
    graph_objs.Bar(
        x=[f[1] for f in to_show],
        y=[f[0] for f in to_show],
        orientation="h"
)]
plotly.offline.iplot({"data":features_importance, "layout":graph_objs.Layout(title="Most important features in the final model",
      margin=graph_objs.Margin(
           l=200,
        pad=3
    ),)})

XGBoost considered custom features as important, so it also confirmed that some non-word features of a text can also be used to predict the sentiment. Most of them were even more important than the actual presence of some emotion-expressing words in the text. 

# Summary
Experiment showed that prediction of text sentiment is a non-trivial task for machine learning. A lot of preprocessing is required just to be able to run any algorithm and see - usually not great - results. 
Main problem for sentiment analysis is to craft the machine representation of the text. Simple bag-of-words was definitely not enough to obtain satisfying results, thus a lot of additional features were created basing on common sense (number of emoticons, exclamation marks etc.). Word2vec representation significantly raised the predictions quality.
I think that a slight improvement in classification accuracy for the given training dataset could be developed, but since it contained highly skewed data (small number of negative cases), the difference will be probably in the order of a few percent.
The thing that could possibly improve classification results will be to add a lot of additional examples (increase training dataset), because given 5971 examples obviously do not contain every combination of words usage, moreover - a lot of emotion-expressing words surely are missing.